# Concrete Network
Assignment submission for Introduction to Deep Learning

**Part A.** Build a baseline model

*Task:* Build a neural network with
- one hidden layer of 10 nodes and ReLU activation functions
- **adam** optimizer and **mean squared error** as loss function

*Note:* Part B. requires repeating this exercise with normalized data.
Therefore, for this part A. we work with the unnormalized raw data.

## Preparation: Loading modules and data
We install required packages, load all required modules, load the data,
and split it into a feature dataframe `features` and a target series
`target`.

In [1]:
# For compatibility, we install the required packages in the same
# version as used in the course labs.
%pip install numpy==2.0.2
%pip install pandas==2.2.2
%pip install tensorflow_cpu==2.18.0
%pip install scikit-learn==1.6.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


We import the required Python modules and load the concrete data from
the specified URL. We also define a custom callback function for Keras
to avoid lengthy outputs in Jupyter notebooks and clear the display
after each epoch.

In [2]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy.typing import ArrayLike
from IPython.display import clear_output

class ClearDisplay(keras.callbacks.Callback):
    """A simple custom callback function for the Keras fitting that
    clears the display before starting a new epoch"""
    def on_epoch_begin(self, epoch, logs=None):
        clear_output()
    def on_train_batch_end(self, batch, logs=None):
        pass

filepath='https://cocl.us/concrete_data'
concrete_data = pd.read_csv(filepath)

concrete_data.describe()

2025-02-17 16:09:51.201989: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


As seen before, the data is already clean and we can proceed isolating
the **Strength** as our target.

In [3]:
target_name = 'Strength'
target = concrete_data[target_name]
features = concrete_data.drop(target_name, axis=1)
features.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


## Build the model

In [4]:
def baseline_model(input_shape: tuple) -> keras.models.Sequential:
    """Build a baseline neural network with Keras.

    The model uses ReLU activation functions, adam optimizer and
    mean squared error as loss function.

    Args:
        input_shape (tuple) : The shape of the input data samples

    Returns:
        Model with one hidden layer of 10 nodes
    """
    model = keras.models.Sequential()
    model.add(keras.layers.Input(input_shape)) # input layer 
    model.add(keras.layers.Dense(10, activation='relu')) # hidden layer
    model.add(keras.layers.Dense(1)) # output layer

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


## 1. Split the data
Now that we have our target data in `target` and the predictor features
in `features`, we split the data into training and testing sets. We use
the `train_test_split` function from *scikit-learn*.

In [5]:
# Since part B explicitly requires normalizing, we are supposed to
# proceed with the raw data without normlization here.

# Split the data into training and testing data:
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    test_size=0.3)

## 2. Train the model
We load our model and train it on the training data using 50 epochs.

In [6]:
input_shape = X_train.iloc[0].shape
model = baseline_model(input_shape) # initialize model
model.fit(X_train, y_train, epochs=50, callbacks=[ClearDisplay()])

Epoch 50/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 540.9747 


## 3. Evaluate the model
We evaluate the model on the test data and compute the mean squared
error between the predicted and actual values for the concrete strength

In [7]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"The mean squared error for the concrete strength is {mse:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
The mean squared error for the concrete strength is 516.2280


Let's look at some values to see how the huge MSE is coming up.
We print the true and predicted values along with the squared error
for the first 10 data points:

In [8]:
for i in range(10):
    print(f"{y_test.iloc[i]:5.2f}, {y_pred[i].item():5.2f},", end=' ')
    print(f"{(y_test.iloc[i] - y_pred[i].item())**2:7.2f}")

10.03, 23.89,  191.99
60.28, 63.94,   13.40
31.12, 24.93,   38.28
 9.69, 15.98,   39.58
21.65, 55.67, 1157.04
54.38, 41.52,  165.37
17.84, 37.65,  392.45
31.64, 20.13,  132.46
32.01, 39.88,   61.89
47.22, 99.00, 2681.25


As we see, the predictions are way off for many values, resulting in huge
squared errors.

## 4. Repeat 50 times
Create a list of 50 mean squared errors by running the split, train,
test cycle repeatedly.

In [9]:
def single_cycle(X: ArrayLike, y: ArrayLike) -> float:
    """Run a single cycle of splitting, training, and testing.
    
    Args:
        X : Feature data
        y : Target data

    Returns:
        The mean squared error for the concrete strength
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    input_shape = X_train.iloc[0].shape
    model = baseline_model(input_shape) # initialize model
    model.fit(X_train, y_train, epochs=50, verbose=0) # fit silently
    y_pred = model.predict(X_test, verbose=0) # predict silently
    return mean_squared_error(y_test, y_pred)

# Run 50 times, creating a list of MSEs:
strength_mses = [single_cycle(features, target) for _ in range(50)]

## 5. Report mean and standard deviation
We calculate the mean and the standard deviation of the list of mean
squared errors for the concrete strengths.

In [10]:
strength_mses = np.array(strength_mses) # convert to numpy array
print(f"The calculated mean squared errors have:")
print(f"  a mean value of         {strength_mses.mean():.4f}")
print(f"  a standard deviation of {strength_mses.std():.4f}")


The calculated mean squared errors have:
  a mean value of         297.7499
  a standard deviation of 234.4015
